# Parte 01 - Feature Backfill

Este material é baseado no tutorial avançado da Hopsworks Feature Store. Você irá trabalhar com dados relacionados ao uso de bicicletas do Citibike e observações meteorológicas na cidade de Nova York.

O objetivo é trabalhar com o **Hopsworks Feature Store** para dados em lote, com o objetivo de treinar e implantar um modelo que possa prever o uso de bicicletas do Citibike por estação no futuro.

## 🗒️ Este notebook está dividido em 3 seções:
1. Carregamento dos dados e Feature Engineering.
2. Conexão ao Hopsworks Feature Store.
3. Criação Feature Groups e enviá-los para o feature store.

## 📝 Importando os Pacotes

In [ ]:
# instalando os pacotes
!pip install -U hopsworks --quiet
!pip install python-dotenv

In [ ]:
# importando pacotes
from datetime import timedelta, datetime
import pandas as pd
import plotly.express as px
import os

from pandas.tseries.holiday import USFederalHolidayCalendar

from features import citibike, meteorological_measurements

# Mutando avisos
import warnings
warnings.filterwarnings("ignore")

---

## 💽 Carregar os dados históricos e 🛠️ Realizar Engenharia de Características

Os dados que você irá utilizar vêm de três fontes diferentes:

- Históricos de viagens do Citi Bike [Trip Histories](https://s3.amazonaws.com/tripdata/index.html);
- Obtenção de Feriados Nacionais dos EUA a partir do `USFederalHolidayCalendar` (pacote `pandas.tseries.holiday`);
- Diferentes observações meteorológicas do [VisualCrossing](https://www.visualcrossing.com/).

### 🚲 Informações sobre o uso do Citibike

Os arquivos baixáveis dos dados de viagens do Citi Bike estão localizados [aqui](https://s3.amazonaws.com/tripdata/index.html). Originalmente, os dados incluem:

| Original em Inglês               | Português                      |
|--------------------------|--------------------------------|
| Ride ID                  | Identificação da Viagem        |
| Rideable type            | Tipo de Veículo                |
| Started at               | Início da Viagem               |
| Ended at                 | Fim da Viagem                  |
| Start station name       | Nome da Estação de Início      |
| Start station ID         | ID da Estação de Início        |
| End station name         | Nome da Estação de Fim         |
| End station ID           | ID da Estação de Fim           |
| Start latitude           | Latitude de Início             |
| Start longitude          | Longitude de Início            |
| End latitude             | Latitude de Fim                |
| End Longitude            | Longitude de Fim               |
| Member or casual ride    | Membro ou Passeio Casual       |



Vamos baixar alguns dados [daqui](https://s3.amazonaws.com/tripdata/index.html) e realizar o pré-processamento (remoção de colunas redundantes e agrupamento de dados).

In [ ]:
# Obtendo os dados
df_raw = citibike.get_citibike_data("01/2023", "04/2023")
df_raw.head(3)

In [ ]:
# Converter a coluna 'station_id' para o tipo string para representação categórica.
df_raw.station_id = df_raw.station_id.astype(str)

In [ ]:
# Feature Engineering do Citibike
df_enhanced = citibike.engineer_citibike_features(df_raw)

# Remover linhas com valores ausentes no DataFrame aprimorado
df_enhanced = df_enhanced.dropna()

# Converter 'station_id' para o tipo string para representação categórica
df_enhanced.station_id = df_enhanced.station_id.astype(str)

# Exibir as três primeiras linhas do DataFrame aprimorado
df_enhanced.head(3)


In [ ]:
# Criando uma amostra de 'station_id' aleatório do DataFrame aprimorado
random_station_id = df_enhanced.station_id.sample(1).values[0]

# Exibir as três primeiras linhas do DataFrame aprimorado para o 'station_id' selecionado aleatoriamente
df_enhanced[df_enhanced.station_id == random_station_id].head(3)


In [ ]:
# Exibir informações sobre o DataFrame, incluindo tipos de dados, contagens não nulas e uso de memória
df_enhanced.info()

### 📒 Informações sobre Estações do Citibike

In [ ]:
# Ler o arquivo CSV contendo informações da estação em um DataFrame
df_stations_info = pd.read_csv("data/stations_info.csv")


In [ ]:
# Remover linhas duplicadas com base na coluna 'station_id' no DataFrame de informações da estação
df_stations_info = df_stations_info.drop_duplicates(subset=["station_id"])

# Resetar o índice do DataFrame e remover qualquer linha com valores ausentes
df_stations_info = df_stations_info.reset_index(drop=True).dropna()

# Converter 'station_id' para o tipo string para representação categórica
df_stations_info.station_id = df_stations_info.station_id.astype(str)

In [ ]:
# Exibir as três primeiras linhas do DataFrame de informações da estação
df_stations_info.head(3)

In [ ]:
# Criar um mapa de dispersão usando Plotly Express com informações da estação
fig = px.scatter_mapbox(
    df_stations_info, 
    lat="lat", 
    lon="long",
    zoom=9.5,
    hover_name="station_name",
    height=400,
    width=600,
)

# Definir o estilo do mapa como 'open-street-map'
fig.update_layout(mapbox_style="open-street-map")

# Ajustar as margens do layout para remover espaço desnecessário
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

# Exibir o mapa
fig.show()

### 📅 Feriados nos Estados Unidos

In [ ]:
# Criar um calendário de feriados federais dos EUA
cal = USFederalHolidayCalendar()

# Gerar uma feature para 20 anos de dias de feriado nos EUA
start_date_for_cal = datetime.strptime('2017-01-01', '%Y-%m-%d')
end_date_for_cal = start_date_for_cal + timedelta(days=365*10)

# Criar um DataFrame com datas de feriados e uma coluna correspondente 'holiday'
holidays = pd.DataFrame(
    cal.holidays(start=start_date_for_cal, end=end_date_for_cal),
    columns=['date'],
)
holidays['date'] = holidays['date'].dt.strftime('%Y-%m-%d')
holidays['holiday'] = 1


In [ ]:
# Criar um DataFrame com um intervalo de datas de start_date_for_cal a end_date_for_cal
df_holidays = pd.DataFrame(
    pd.date_range(start_date_for_cal, end_date_for_cal),
    columns=["date"],
)

# Formatar a coluna 'date' para corresponder ao formato '%Y-%m-%d'
df_holidays['date'] = df_holidays['date'].dt.strftime('%Y-%m-%d')

# Exibir as três primeiras linhas do DataFrame
df_holidays.head(3)


In [ ]:
# Definir a coluna 'date' como índice e unir o DataFrame 'holidays' na coluna 'date'
# Preencher valores ausentes com 0 após a união
df_holidays = df_holidays.set_index("date").join(
    holidays.set_index("date"), 
    how="left",
).fillna(0)

In [ ]:
# Converter a coluna 'holiday' para o tipo inteiro
df_holidays['holiday'] = df_holidays['holiday'].astype(int)

# Resetar o índice, trazendo a coluna 'date' de volta como uma coluna regular
df_holidays = df_holidays.reset_index(drop=False)

# Exibir as três primeiras linhas do DataFrame
df_holidays.head(3)


In [ ]:
# Verificando o DataFrame
df_holidays.tail(3)

### 🌤 Medidas Meteorológicas do VisualCrossing

Você irá analisar os dados meteorológicos, então você deve obter uma chave de API do [VisualCrossing](https://www.visualcrossing.com/). Você pode usar [este link](https://www.visualcrossing.com/weather-api).

#### Não se esqueça de criar um arquivo de configuração `.env` dentro deste diretório, onde todas as variáveis de ambiente necessárias serão armazenadas:

`WEATHER_API_KEY = "SUA_CHAVE_DE_API"`

> Se você fizer isso depois de executar este notebook, reinicie o Kernel do Python (porque `functions.py` não terá essas variáveis em seu namespace).

![](images/api_keys_env_file.png)

In [ ]:
# Converter a coluna 'date' para o tipo string
df_enhanced.date = df_enhanced.date.astype(str)

# Encontrar as datas mínima e máxima na coluna 'date'
start_date, end_date = df_enhanced.date.min(), df_enhanced.date.max()


In [ ]:
# Obter dados meteorológicos para a cidade de Nova York dentro do intervalo de datas especificado
df_weather = meteorological_measurements.get_weather_data(
    city="nyc",
    start_date=str(start_date).split()[0],
    end_date=str(end_date).split()[0],
)
df_weather.tail(3)


In [ ]:
# Criação das colunas Unix
df_enhanced["timestamp"] = df_enhanced["date"].apply(
    meteorological_measurements.convert_date_to_unix
)
df_holidays["timestamp"] = df_holidays["date"].apply(
    meteorological_measurements.convert_date_to_unix
)
df_weather["timestamp"] = df_weather["date"].apply(
    meteorological_measurements.convert_date_to_unix
)


---

##  📡 Conectando ao Hopsworks Feature Store

In [ ]:
# Importando os pacotes
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## 🪄 Criando Grupos de Características

Um [feature group](https://docs.hopsworks.ai/3.0/concepts/fs/feature_group/fg_overview/) pode ser visto como uma coleção de características conceitualmente relacionadas. Neste caso, você irá criar os seguintes grupos de características: Uso do CitiBike por estação, Informações das Estações, Medidas Meteorológicas em NYC e Feriados nos EUA.

Uma vez criado o Feature Group, basta fazer as inserções.

`fs.get_or_create_feature_group`: Este método é utilizado para obter ou criar um feature group no **Hopsworks Feature Store**. Neste caso, está sendo criado um feature group chamado "citibike_usage". Os parâmetros incluem o nome, versão, descrição, chave primária (composta por "date" e "station_id") e o tempo do evento definido como "timestamp".

`citibike_usage_fg.insert`: Este método insere dados no grupo de características 'citibike_usage'. O DataFrame df_enhanced é utilizado como fonte de dados. O parâmetro write_options={"wait_for_job": True} indica que o código deve esperar pela conclusão da inserção antes de continuar.

In [ ]:
# Criando o Feature Group citibike_usage_fg
citibike_usage_fg = fs.get_or_create_feature_group(
    name="citibike_usage",
    version=1,
    description="Citibike stations usage data.",
    primary_key=["date", "station_id"],
    event_time="timestamp",
)

In [ ]:
# Inserindo no Feature Group citibike_usage_fg
citibike_usage_fg.insert(
    df_enhanced,
    write_options={"wait_for_job": True},
)

In [ ]:
# Criando o Feature Group citibike_stations_info_fg
citibike_stations_info_fg = fs.get_or_create_feature_group(
    name="citibike_stations_info",
    version=1,
    description="Citibike stations information.",
    primary_key=['station_id'],
)

In [ ]:
# Inserindo no Feature Group citibike_stations_info_fg
citibike_stations_info_fg.insert(
    df_stations_info,
    write_options={"wait_for_job": True},
)

In [ ]:
# Criando o Feature Group us_holidays_fg
us_holidays_fg = fs.get_or_create_feature_group(
    name="us_holidays",
    version=1,
    description="US holidays calendar.",
    primary_key=["date"],
    event_time="timestamp",
)

In [ ]:
# Inserindo no Feature Group us_holidays_fg
us_holidays_fg.insert(
    df_holidays,
    write_options={"wait_for_job": True},
)

In [ ]:
# Criando o Feature Group meteorological_measurements_fg
meteorological_measurements_fg = fs.get_or_create_feature_group(
    name="meteorological_measurements",
    version=1,
    description="Meteorological measurements for NYC.",
    primary_key=["date"],
    event_time="timestamp",
)

In [ ]:
# Inserindo no Feature Group meteorological_measurements_fg
meteorological_measurements_fg.insert(
    df_weather, 
    write_options={"wait_for_job": True},
)

## ⏭️ **Próxima Aula:** Parte 02: Feature Pipeline

No próximo notebook, você estará analisando novos dados mensais para os Feature Groups.
